# Callbacks

Callbacks in LangChain provide a way to hook into various stages of the LLM application's execution. These allow you to monitor or modify the behavior of chains, tools, or LLMs during runtime. They are especially useful for logging, debugging, or triggering actions based on certain events in the execution process

In LangChain, callbacks can be passed at runtime to capture events during the execution of chains, tools, or models. This approach simplifies the process by allowing you to pass the callback handlers once, and they will automatically be used for all nested objects involved in the execution. This eliminates the need to manually attach the handlers to each individual component, like tools or models.

# Example: Passing Callbacks at Runtime

In [3]:
from typing import Any, Dict, List
from langchain_openai import ChatOpenAI

from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate


class LoggingHandler(BaseCallbackHandler):
    # This callback handler will log when the chat model starts
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    # This callback handler will log when the LLM finishes execution
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    # This callback handler will log when a chain starts
    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        # Adding a check for None to avoid the error
        chain_name = serialized.get('name') if serialized else 'Unknown Chain'
        print(f"Chain {chain_name} started")

    # This callback handler will log when a chain ends
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")


# Instantiate the callback handler
callbacks = [LoggingHandler()]

# Set up the LLM (ChatOpenAI) and prompt template
llm = ChatOpenAI(model="gpt-4")  # Change to your desired model
prompt = ChatPromptTemplate.from_template("What is 1 + {number}?")

# Combine the prompt and LLM into a chain
chain = prompt | llm

# Invoke the chain, passing the callback handlers as part of the config
chain.invoke({"number": "2"}, config={"callbacks": callbacks})


Chain Unknown Chain started
Chain ChatPromptTemplate started
Chain ended, outputs: messages=[HumanMessage(content='What is 1 + 2?', additional_kwargs={}, response_metadata={})]
Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='3', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-20324a36-0e9e-40cd-a14b-135ab1c3e5cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details':

AIMessage(content='3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-20324a36-0e9e-40cd-a14b-135ab1c3e5cc-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# How to attach callbacks to a runnable

To attach callbacks to a Runnable in LangChain, you can use the .with_config() method, which allows you to bind runtime configuration, such as callbacks, to your Runnable. This makes it easier to reuse the callbacks across multiple invocations without passing them in each time

In [4]:
from typing import Any, Dict, List
from langchain_openai import ChatOpenAI
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult
from langchain_core.prompts import ChatPromptTemplate


class LoggingHandler(BaseCallbackHandler):
    # Callback to log when the chat model starts
    def on_chat_model_start(
        self, serialized: Dict[str, Any], messages: List[List[BaseMessage]], **kwargs
    ) -> None:
        print("Chat model started")

    # Callback to log when the LLM finishes execution
    def on_llm_end(self, response: LLMResult, **kwargs) -> None:
        print(f"Chat model ended, response: {response}")

    # Callback to log when a chain starts
    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs
    ) -> None:
        # Check if serialized is not None to prevent errors
        chain_name = serialized.get('name') if serialized else 'Unknown Chain'
        print(f"Chain {chain_name} started")

    # Callback to log when a chain ends
    def on_chain_end(self, outputs: Dict[str, Any], **kwargs) -> None:
        print(f"Chain ended, outputs: {outputs}")


# Instantiate the callback handler
callbacks = [LoggingHandler()]

# Set up the LLM and prompt template
llm = ChatOpenAI(model="gpt-4")  # Replace with the desired model
prompt = ChatPromptTemplate.from_template("What is 1 + {number}?")

# Combine the prompt and LLM into a chain
chain = prompt | llm

# Attach the callbacks using the .with_config() method
chain_with_callbacks = chain.with_config(callbacks=callbacks)

# Invoke the chain with the given input
chain_with_callbacks.invoke({"number": "2"})


Chain Unknown Chain started
Chain ChatPromptTemplate started
Chain ended, outputs: messages=[HumanMessage(content='What is 1 + 2?', additional_kwargs={}, response_metadata={})]
Chat model started
Chat model ended, response: generations=[[ChatGeneration(text='3', generation_info={'finish_reason': 'stop', 'logprobs': None}, message=AIMessage(content='3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5076244f-c01f-4adb-b0fc-b19299add7fb-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details':

AIMessage(content='3', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5076244f-c01f-4adb-b0fc-b19299add7fb-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

# Key Points:
1. Creating the Callback Handler: The LoggingHandler class is derived from BaseCallbackHandler and overrides various callback methods such as on_chat_model_start, on_llm_end, on_chain_start, and on_chain_end to print messages when certain events occur during execution.

2. Attaching the Callbacks: The .with_config(callbacks=callbacks) method attaches the LoggingHandler to the chain. This ensures that all callbacks specified in the callbacks list will be triggered during the execution of the chain.

3. Invocation of the Chain: When chain_with_callbacks.invoke({"number": "2"}) is called, the callbacks are automatically triggered during the execution.


Expected Output:

This will produce log messages for each stage in the process, such as:

"Chain started"

"Chat model started"

"Chain ended"

"Chat model ended, response: [response data]"